# Try different models-compare clustering results over eval. set
TODO: try different eps

In [9]:
# load eval.set (duplicates free)
import json
filename = 'kmeans_negative_test_0noise.json'
#filename = '/home/nick/anaconda3/bin/Tirocinio/doc2vec_tryouts/EvaluateModels/english_4_clusters.json'
#filename = '/home/nick/anaconda3/bin/Tirocinio/doc2vec_tryouts/EvaluateModels/pre-clustered_docs_harder.json'
with open(filename, 'r') as file:
    cdocs = json.load(file)
print("#Number of clusters:",len(cdocs))
print("##Cluster length:",[str(len(cluster)) for cluster in cdocs])
print("### Doc length: ",[len(doc['headline']+doc['bodyText']) for cluster in cdocs for doc in cluster 
                          if not(doc['bodyText'] is None)])
#print("### Doc length: ",[len(doc['title']+doc['abstract']) for cluster in cdocs for doc in cluster])

#Number of clusters: 2
##Cluster length: ['3', '20']
### Doc length:  [10558, 3702, 2439, 6075, 1886, 19646, 4117, 1547, 2014, 6584, 10788, 2512, 2375, 3694, 4614, 2758, 5397, 5287, 9861, 3475, 4289, 4561, 6182]


## Title+abstract models

In [10]:
# train different models
from gensim.utils import simple_preprocess as sp
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import multiprocessing

unwrapped_docs = [doc for cluster in cdocs for doc in cluster]
# title + abstract models
train_corpus_ta = [ TaggedDocument(sp(doc['headline'] + doc['bodyText']) ,[i]) for i, doc in enumerate(unwrapped_docs)]
#train_corpus_ta = [ TaggedDocument(sp(doc['title'] + doc['abstract']) ,[i]) for i, doc in enumerate(unwrapped_docs)]
epochs = 45
vec_size = 100
models = [
    # dm = 0, simple SG, simpler model, most of the time efficient and accurate
    Doc2Vec(dm=0, vector_size=500, negative=5, hs=0, min_count=2, sample=0, 
            epochs=45, workers=multiprocessing.cpu_count(), comment='dm=0, vec = 500'),
    Doc2Vec(dm=0, vector_size=800, negative=5, hs=0, min_count=2, sample=0, 
            epochs=45, workers=multiprocessing.cpu_count(), comment='dm=0, vec = 800'),
    Doc2Vec(dm=0, vector_size=800, negative=12, hs=0, min_count=2, sample=0, 
            epochs=40, workers=multiprocessing.cpu_count(), comment='dm=0, vec = 800, negative=12'),
]

# build our vocabulary of words (all the unique words encountered inside our corpus)
for model in models:
    model.build_vocab(train_corpus_ta)
    print(model, "-VocabSize:", len(model.wv.vocab))
print("Vocabulary created!")

# train the models on the given data!
counter = 0
for model in models:
    print("Training %s" % model)
    %time model.train(train_corpus_ta, total_examples=len(train_corpus_ta), epochs=model.epochs)

Doc2Vec("dm=0, vec = 500",dbow,d500,n5,mc2,t4) -VocabSize: 1875
Doc2Vec("dm=0, vec = 800",dbow,d800,n5,mc2,t4) -VocabSize: 1875
Doc2Vec("dm=0, vec = 800, negative=12",dbow,d800,n12,mc2,t4) -VocabSize: 1875
Vocabulary created!
Training Doc2Vec("dm=0, vec = 500",dbow,d500,n5,mc2,t4)
CPU times: user 3 s, sys: 0 ns, total: 3 s
Wall time: 1.61 s
Training Doc2Vec("dm=0, vec = 800",dbow,d800,n5,mc2,t4)
CPU times: user 4.08 s, sys: 0 ns, total: 4.08 s
Wall time: 2.1 s
Training Doc2Vec("dm=0, vec = 800, negative=12",dbow,d800,n12,mc2,t4)
CPU times: user 8.28 s, sys: 0 ns, total: 8.28 s
Wall time: 4.1 s


# KMeans
## Choosing K (highlight the right k in red)

In [14]:
from sklearn.cluster import KMeans
titles = [doc['headline'] for doc in unwrapped_docs]
#titles = [doc['title'] for doc in unwrapped_docs]
# let's try Kmeans with K between 2 and 20
def run_KMeans_K_times(model, K):
    doc_vecs = [model.docvecs[j] for j in range(len(model.docvecs))]
    sse = []
    kmeans_results = []
    # run k means with k between 2 and K
    for i in range(2, K+1):
        kmeans = KMeans(n_clusters=i, init='k-means++', # smart-way of choosing starting point
               n_init= 50, # run k-means 100 times and get the best result in terms of SSE
                n_jobs = -1 # use every core
        )
        # compute sse for this kmeans result
        kmeans.fit(doc_vecs)
        sse.append(kmeans.inertia_)
        
        # get 'visual' clustering, by saving a list of titles
        clusters = {label: [] for label in kmeans.labels_}
        for label, list_ in clusters.items():
            for i, l in enumerate(kmeans.labels_):
                if label==l:
                    clusters[label].append(titles[i])
        
        kmeans_results.append(clusters)
    return kmeans_results, sse

import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
# using my api-key
tls.set_credentials_file(username='D4nt3_', api_key='4O71urldgOueVtcApOdX')
graph_name = 'kmeans_choosing_k'

kmeans_results, sse = run_KMeans_K_times(models[1], 23)
K = 20 # max_k
# plot results of K and SSE
trace = go.Scatter(
    x = [k for k in range(2, K)],
    y = sse,
    mode = 'lines+markers'
)
layout = dict(title = 'Choosing K in Kmeans',
              xaxis = dict(title = 'K value'),
              yaxis = dict(title = 'SSE'),
              )

data = [trace]
fig = dict(data=data, layout=layout)
py.iplot(fig, filename = graph_name)

# Evaluate Kmeans
On KMeans best conditions: 4 clusters, no-noise, equal-size in clusters, (approximately) same density, (unsure about shape).


In [15]:
import utils
import my_dbscan
import model_evaluation as me

clusters = []
models_precision_kmeans = []
models_recall_kmeans = []
expected_clusters = []
for docs in cdocs:
    #expected_clusters.append([doc['title'] for doc in docs])
    expected_clusters.append([doc['headline'] for doc in docs])
        
for model in models:
    # K already chosen before, with value 4
    cluster_results, sse = run_KMeans_K_times(model, 2)
    #cluster_results, sse = run_KMeans_K_times(model, 10)
    cluster_results = cluster_results[-1] # get 4-means results
    clist = []
    for key, cluster in cluster_results.items():
        clist.append(cluster)
    
    clusters.append(clist)
    
     # evaluate clustering
    precision, recall = me.compute_clustering_accuracy(clist, expected_clusters)
    print('Precision score: %s, Recall score: %s'%(precision, recall))
    
    models_precision_kmeans.append(precision)
    models_recall_kmeans.append(recall)
    # last check: we want to penalize models that simply cluster all docs together (that's not a valid result)
    # that's way we have precision score
    print("#Number of clusters found: {0}, against number of pre-computed clusters: {1}#\n".format(
        len(clist), len(cdocs)))

Results occurences(correct guess, cluster index):  [(3, 0), (16, 1)]
Accuracy (Precision) over each cluster:  [42.857142857142854, 100.0]
Accuracy (Recall) over each cluster:  [100.0, 80.0]
Precision score: 71.42857142857143, Recall score: 90.0
#Number of clusters found: 2, against number of pre-computed clusters: 2#

Results occurences(correct guess, cluster index):  [(2, 0), (18, 0)]
Accuracy (Precision) over each cluster:  [10.0, 90.0]
Accuracy (Recall) over each cluster:  [66.66666666666667, 90.0]
Precision score: 50.0, Recall score: 78.33333333333334
#Number of clusters found: 2, against number of pre-computed clusters: 2#

Results occurences(correct guess, cluster index):  [(3, 0), (11, 0)]
Accuracy (Precision) over each cluster:  [21.428571428571427, 78.57142857142857]
Accuracy (Recall) over each cluster:  [100.0, 55.0]
Precision score: 50.0, Recall score: 77.5
#Number of clusters found: 2, against number of pre-computed clusters: 2#



In [36]:
print(clusters[1])

[['First woman: Smithsonian Air and Space director looks from the moon to Mars', 'The first human on Mars should be a woman – we deserve stardust too', 'Forgotten heroes of the first world war', 'Overlord review – nasty second world war action-horror fantasy', 'Reflections on the first world war and Armistice Day', 'Thousands of Ypres sculptures to commemorate world war one dead', 'Black soldiers’ role in the first world war', 'Second world war winners and the appeal of Dad’s Army', 'Transcription by Kate Atkinson review – second world war spying hijinks', 'Bells will ring out: world to mark end of first world war, 100 years on', 'Second world war pilot Mary Ellis dies aged 101', 'London Sinfonietta/George Benjamin\xa0review – austere first world war meditation', 'Long-lost photo album of first world war soldier given to his family', 'Empire Cinemas rejects first world war short film by Bible Society', 'When heroes of the first world war made playing fields out of battlefields', 'Relie

# Evaluate Clustering DBSCAN

In [37]:
import utils
import my_dbscan
import model_evaluation as me

models_recall = []
models_precision = []
clusters_found = [] # keep clusters results, they're useful later on
min_s = 2 # min_samples
titles = [doc['headline'] for doc in unwrapped_docs]
urls = titles
expected_clusters = []
for docs in cdocs:
    expected_clusters.append([doc['headline'] for doc in docs])
#urls = [doc['url'] for doc in unwrapped_docs]
for k, model in enumerate(models):
    # try different eps for some model
    if k == 1:
        eps = 0.41
        eps_increment = 0.13
        min_s = 2
    elif k==6:
        eps = 0.11
        eps_increment = 0.13
        min_s = 3
    else:
        eps = 0.11
        eps_increment = 0.13
        min_s = 2
    # get list of document vectors
    doc_vecs = [model.docvecs[j] for j in range(len(model.docvecs))]
    
    # apply dbscan clustering to these vectors
    titles_clusters = my_dbscan.apply_dbscan(doc_vecs = doc_vecs, titles = titles, 
                                               urls = urls, subset_length = len(titles),
                                                 eps = eps, eps_increment = eps_increment,
                                               n_iterations = 3, verbose = False, min_samples = min_s)
    # get clusters as list of titles
    #titles_clusters = utils.getDocTitleFromUrl(unwrapped_docs, urls_cluster_list)
    clusters_found.append(titles_clusters)
    
    # evaluate clustering

    precision, recall = me.compute_clustering_accuracy(titles_clusters, expected_clusters)
    print('Precision score: %s, Recall score: %s'%(precision, recall))
    
    models_precision.append(precision)
    models_recall.append(recall)
    # last check: we want to penalize models that simply cluster all docs together (that's not a valid result)
    # that's way we have precision score
    print("#Number of clusters found: {0}, against number of pre-computed clusters: {1}#\n".format(
        len(titles_clusters), len(cdocs)))


Results occurences(correct guess, cluster index):  [(0, 3), (3, 3)]
Accuracy (Precision) over each cluster:  [0, 100.0]
Accuracy (Recall) over each cluster:  [0, 15.0]
Precision score: 50.0, Recall score: 7.5
#Number of clusters found: 4, against number of pre-computed clusters: 2#

Results occurences(correct guess, cluster index):  [(3, 0), (11, 1)]
Accuracy (Precision) over each cluster:  [100.0, 100.0]
Accuracy (Recall) over each cluster:  [100.0, 55.0]
Precision score: 100.0, Recall score: 77.5
#Number of clusters found: 4, against number of pre-computed clusters: 2#

Results occurences(correct guess, cluster index):  [(2, 2), (7, 3)]
Accuracy (Precision) over each cluster:  [100.0, 100.0]
Accuracy (Recall) over each cluster:  [66.66666666666667, 35.0]
Precision score: 100.0, Recall score: 50.833333333333336
#Number of clusters found: 4, against number of pre-computed clusters: 2#



In [38]:
for cluster in clusters_found[1]:
    print(cluster)


['First woman: Smithsonian Air and Space director looks from the moon to Mars', 'The first human on Mars should be a woman – we deserve stardust too', "Trump's moon shot might be steered by a woman, says Nasa chief"]
['Forgotten heroes of the first world war', 'Overlord review – nasty second world war action-horror fantasy', 'Reflections on the first world war and Armistice Day', 'Black soldiers’ role in the first world war', 'Second world war winners and the appeal of Dad’s Army', 'Transcription by Kate Atkinson review – second world war spying hijinks', 'Second world war pilot Mary Ellis dies aged 101', 'London Sinfonietta/George Benjamin\xa0review – austere first world war meditation', 'Trump to meet Putin and Erdoğan during first world war ceremonies', 'Relief and reckoning: the first world war was over and Australia counted the cost', '11-11: Memories Retold review – a first world war game in which no shots are fired']
['Thousands of Ypres sculptures to commemorate world war one d

In [35]:
import utils
#doc_vecs = [models[1].docvecs[j] for j in range(len(models[1].docvecs))]
# finally, call eps-estimate function
title_dist_tuples = utils.choose_eps(2, doc_vecs, titles)
data = utils.visualize_eps_graph(title_dist_tuples=title_dist_tuples)
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.tools as tls
import plotly.graph_objs as go

tls.set_credentials_file(username='D4nt3_', api_key='4O71urldgOueVtcApOdX')
py.iplot(data, filename='step-graph-eps')

# Visualize results graphically

In [8]:
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
# using my api-key
tls.set_credentials_file(username='D4nt3', api_key='FdMB4O6qCfciGDOnLvdQ')
graph_name = 'model_eval_results'

model_descr = [model.comment for model in models]
# plot test-accuracy results, plus silhoutte scores (times 100, since all the values are in between 0-1)
trace0 = go.Bar(
    x = model_descr + ['loaded model'],
    y = models_precision,
    name='Precision',
    marker=dict(
        color='rgb(49,130,189)'
    )
)
trace1 = go.Bar(
    x = model_descr + ['loaded model'],
    y = [s*100 for s in s_scores] + [0],
    name='Silhoutte score',
    marker=dict(
        color='rgb(204,50,100)',
    )
    
)

trace2 = go.Bar(
    x = model_descr + ['loaded model'],
    y = models_recall,
    name='Recall',
    marker=dict(
        color='rgb(155, 244, 66)',
    )
    
)
# precision, recall, silhoutte
data = [trace0, trace2,  trace1]
layout = go.Layout(
    title = 'Clustering Accuracy Results',
    xaxis=dict(
        tickfont=dict(
            size=10,
            color='rgb(107, 107, 107)',
            
        ),
        tickangle = -45
    ),
    yaxis=dict(
        title='Clustering accuracy (%)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
   
    barmode='group',
    bargap=0.2,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename=graph_name)

# Silhoutte score
Silhouette coefficient combines ideas of both cohesion and separation, 
but for individual points, as well as clusters and 
clusterings; the Silhouette Coefficient is calculated using the mean intra-cluster distance (a) and the mean nearest-cluster distance (b) for each sample.
The Silhoutte score is merely the average of each silhoutte coefficient, computed over each sample.
It's a measure of the goodness of clustering, by assuming the fact that a cluster X is defined good if both every sample inside it is close to each other, and far from any other relatively-near cluster Y.
This is not necessarily true for every shape of cluster.

    The best value is 1 and the worst value is -1. Values near 0 indicate overlapping clusters.

In [7]:
from sklearn.metrics import silhouette_score
s_scores = []
for model in models:
    # get clusters for each model
    doc_vecs = [model.docvecs[j] for j in range(len(model.docvecs))]
    titles = [doc['title'] for doc in unwrapped_docs]
    urls = [doc['url'] for doc in unwrapped_docs]
    # apply dbscan clustering to these vectors
    urls_cluster_list = my_dbscan.apply_dbscan(doc_vecs = doc_vecs, titles = titles, 
                                               urls = urls, subset_length = len(titles),
                                                 eps = 0.11, eps_increment = 0.13, n_iterations = 3, verbose = False)
    
    # get cluster labels, mantaining original docs ordering
    labels = []
    for doc in unwrapped_docs:
        noise = True
        for i, url_list in enumerate(urls_cluster_list):
            if(doc['url'] in url_list):
                labels.append(i) # keep cluster id
                noise = False
        if noise:
            labels.append(-1)
    # make sure they have the same size
    assert len(labels) == len(doc_vecs)
    ss = silhouette_score(doc_vecs, labels , metric='cosine')
    s_scores.append(ss)
    print(ss)

0.12614417
0.19104731
0.1858033
0.15527193
0.1609296
-0.023030862
0.17550677
0.19581276


# Conclusions: 
A really low silhoutte score helps us identify models that tend to have very few cluster, hence not really recognizing differences between docs. This is fundamental, since test rules used so far prevent us from recognizing these kinds of models.
Since silhoutte score is an average of silhoutte coefficients, smaller changes of values here may mean greater differences.